In [268]:
import string
import math
import numpy as np

In [269]:
docs = []
query = ""
stopwords = set(['is', 'was', 'the', 'are', 'a', 'an'])

In [270]:
# "C:\D drive\PSG\sem8\20XW86-IR-LAB\vsm-practice-ca\documents\d1.txt"

In [271]:
for i in range(1, 5):
    f = open(f"C:/D drive/PSG/sem8/20XW86-IR-LAB/vsm-practice-ca/documents/d{i}.txt", 'r')
    docs.append(f.read())

In [272]:
f = open(f"C:/D drive/PSG/sem8/20XW86-IR-LAB/vsm-practice-ca/documents/query.txt", 'r')
query = f.read()

In [273]:
docs

['Reuters, dataset, Reuters, benchmark, dataset',
 'Reuters, dataset, class, Reuters, class',
 'document, _multiple, dataset, document',
 'Reuters document, document, class']

In [274]:
query

'Reuters, document, dataset'

In [275]:
processed_docs = []

In [276]:
def transform_text(text):
    text = text.split()
    y = []
    
    for word in text:
        char_list = []
        for c in word:
            if c not in string.punctuation:
                char_list.append(c)
        
        lcase_word = ''.join(char_list[:]).lower()
        
        if lcase_word.isalnum() and lcase_word not in stopwords:
            y.append(lcase_word)

    return ' '.join(y)

In [277]:
transform_text(docs[0])

'reuters dataset reuters benchmark dataset'

In [278]:
for doc in docs:
    processed_docs.append(transform_text(doc))

In [279]:
processed_docs

['reuters dataset reuters benchmark dataset',
 'reuters dataset class reuters class',
 'document multiple dataset document',
 'reuters document document class']

In [280]:
# TF-IDF score calc.
# Wij = (TFk/LENk)*log(N+1/DFk+0.5)
# TFi - term freq within doc.
# LENi - len of document.
# N - total docs
# DFk - document frequency of the word K.

In [281]:
def TF(word, tf_map):
    return tf_map[word]

def DF(word, corpus_docmaps):
    count = 0
    
    for hm in corpus_docmaps:
        if word in hm:
            count += 1
    
    return count

def calculate_weights(doc, wordcount_map, corpus_docmaps):
    vector = [0]*len(wordcount_map)
    
    tf_map = {}
    
    for i in doc.split():
        if i in tf_map:
            tf_map[i] +=1
        else:
            tf_map[i] = 1
    
    for word in doc.split():
        TFik = TF(word, tf_map)
        LENi = sum([tf_map[key] for key in tf_map])
        N = len(corpus_docmaps)
        DFk = DF(word, corpus_docmaps)
        
        try:
            index = list(wordcount_map.keys()).index(word)
            vector[index] = (TFik/LENi)*np.log10((N+1)/(0.5+DFk))
        except ValueError:
            pass
    
    return vector

In [282]:
word_countmap = {}

for doc in processed_docs:
    for word in doc.split():
        if word in word_countmap:
            word_countmap[word] += 1
        else:
            word_countmap[word] = 1

corpus_docmaps = []

for doc in processed_docs:
    hm = {}
    
    for word in doc.split():
        if word in hm:
            hm[word] += 1
        else:
            hm[word] = 1
    
    corpus_docmaps.append(hm)


doc_matrix = []
for doc in processed_docs:
    vector = calculate_weights(doc, word_countmap, corpus_docmaps)
    doc_matrix.append(vector)
    
print(doc_matrix)

[[0.061960783994297275, 0.061960783994297275, 0.10457574905606754, 0, 0, 0], [0.061960783994297275, 0.030980391997148637, 0, 0.12041199826559248, 0, 0], [0, 0.0387254899964358, 0, 0, 0.1505149978319906, 0.1307196863200844], [0.0387254899964358, 0, 0, 0.0752574989159953, 0.1505149978319906, 0]]


In [283]:
query_vector = calculate_weights(transform_text(query), word_countmap, corpus_docmaps)

In [284]:
query_vector

[0.05163398666191439, 0.05163398666191439, 0, 0, 0.10034333188799373, 0]

In [285]:
def cosine_similarity(vec1, vec2):
    dot_prod = sum([x*y for x,y in zip(vec1, vec2)])
    mag1 = sum([x*x for x in vec1])**0.5
    mag2 = sum([y*y for y in vec2])**0.5

    return dot_prod/mag1*mag2

In [286]:
res = []

In [287]:
for i,dv in enumerate(doc_matrix):
    res.append([i+1,cosine_similarity(dv, query_vector)])

0.006398564588646609 0.13643447070668593 0.12410044887747008
0.004798923441484956 0.13891714353454726 0.12410044887747008
0.017102727815528154 0.20308142342284646 0.12410044887747008
0.017102727815528154 0.17267923815826622 0.12410044887747008


In [288]:
sorted(res, key= lambda x:x[1], reverse=True)

[[4, 0.01229132246339271],
 [3, 0.010451257250235832],
 [1, 0.005820118138103466],
 [2, 0.004287077448211349]]